In [0]:
import allennlp
import pandas as pd
import numpy as np
from allennlp.predictors.predictor import Predictor
from google.colab import files
import io

In [7]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo-model-2018.11.30-charpad.tar.gz")

100%|██████████| 765510871/765510871 [00:17<00:00, 42982464.25B/s]
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [0]:
def find_study_type(abstract):
	answers=predictor.predict(passage=abstract, question='Is it vitro, vivo or both?')
	study_type=answers['best_span_str']
	return study_type

In [0]:
# df = pd.read_csv('Users/smruthiramesh/Desktop/IBM-research-challenge/relevant_50_complete.csv')
answers = []
for index,row in df.iterrows():
	abstract_val=row['abstract']
	answers.append(find_study_type(abstract_val))

In [16]:
uploaded = files.upload()

Saving relevant_50_complete.csv to relevant_50_complete (1).csv


In [0]:
df = pd.read_csv(io.BytesIO(uploaded['relevant_50_complete.csv']))

In [0]:
corpus = list(df['abstract'])

In [0]:
vivo_checks = ["mouse", "mice", "rat", "rats", "animal","vivo"]
vitro_checks = ["vitro"]

In [0]:
def classify_study_type(text):

  words = text.split()
  ans = None

  vivo_count = 0
  vitro_count = 0

  for word in vivo_checks:
    if word in words:
      vivo_count+=1
  for word in vitro_checks:
    if word in words:
      vitro_count+=1

  if vivo_count == 0 and vitro_count > 0:
    ans = "vitro"
  elif vivo_count > 0 and vitro_count == 0:
    ans = "vivo"
  elif vivo_count - vitro_count >= 3:
    ans = "vivo"
  elif vitro_count - vivo_count >= 3:
    ans = "vitro"
  else:
    ans = allen_classifier(text) 
  return ans

  # #if it's a close call between vivo and vitro
  # if vivo_count - vitro_count <= 2:
  #   checked_ans = find_study_type(abstract)
 
  # elif vitro_count < vivo_count:
  #   checked_ans = "vivo"
  # elif vitro_count == vivo_count:
  #   checked_ans = "both"
  # else:
  #   checked_ans = "vitro"
  # return checked_ans

In [0]:
def allen_classifier(abstract):
  cleaned_answer = None
  answer = find_study_type(abstract)
  if ("vivo" in answer and "vitro" in answer) or ("both" in answer):
    cleaned_answer = "both"
  elif "vivo" in answer:
    cleaned_answer = "vivo"
  return cleaned_answer

In [0]:
preds = []
for abstract in corpus:
  preds.append(classify_study_type(abstract))

In [0]:
true = [x.lower() for x in df['study type']]
true = [x.split()[1] for x in true]
true = ["both" if "/" in x else x for x in true]

In [176]:
true

['vitro',
 'vivo',
 'both',
 'vivo',
 'both',
 'both',
 'vivo',
 'both',
 'both',
 'both',
 'both',
 'vitro',
 'both',
 'both',
 'both',
 'vitro',
 'both',
 'both',
 'vitro',
 'both',
 'both',
 'vivo']

In [171]:
preds

['vivo',
 'vivo',
 'vivo',
 'vivo',
 'both',
 'both',
 'vivo',
 'both',
 'vivo',
 'both',
 None,
 None,
 None,
 'vivo',
 'vivo',
 'both',
 None,
 None,
 'vitro',
 'both',
 'vivo',
 'vivo']

In [0]:
count = 0
for i in range(len(preds)):
  if true[i] == preds[i]:
    count+=1

In [181]:
count

10

In [182]:
count/len(preds)

0.45454545454545453

Was the current study tested:
 - on animals/in vivo
 - in labware/in vitro
 - or both?